In [1]:
import torch 
import torch.nn as nn
import fasttext as ft
import math

In [ ]:
# ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz -P {path}
! wget -c https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.bin.gz
# !gunzip cc.en.300.bin cc.en.300.bin.gz
!gunzip cc.fr.300.bin cc.fr.300.bin.gz

In [ ]:
en = ft.load_model(str('cc.en.300.bin'))
fr = ft.load_model(str('cc.fr.300.bin'))

# Creating the Methods

In [38]:
d_model = 10 #dimension of word embedding
new_dim = 5

In [39]:
def getWordVectors(sentence):
    sentence = sentence.split(' ')
    vecs = torch.rand((len(sentence),d_model))
    return vecs

In [40]:
def PositionalEncoding(wordVecs):
    for pos in range(wordVecs.shape[0]):
        for i in range(wordVecs[pos].shape[0]):
            if i%2 == 0:
                wordVecs[pos][i] = wordVecs[pos][i] + math.sin(pos/(10000**(2*i/d_model)))
            else:
                wordVecs[pos][i] = wordVecs[pos][i] + math.cos(pos/(10000**(2*i/d_model)))
                
    return wordVecs
                                           

In [41]:
def get_qkv_weights(r,c):
    query_weights = torch.rand((r,c))
    key_weights = torch.rand((r,c))
    value_weights = torch.rand((r,c))
    return query_weights, key_weights, value_weights
    

In [42]:
def qkvs(vectorMatrix, new_dim):
    
    query_weights, key_weights, value_weights = get_qkv_weights(d_model,new_dim)
    
    return torch.matmul(vectorMatrix, query_weights), torch.matmul(vectorMatrix, key_weights), \
    torch.matmul(vectorMatrix, value_weights) 

# Check for transposeness in matrix multiplication

In [43]:
def qk_dotproducts(queries, keys):
    dotproduct_matrix = torch.Tensor([])
    for i in queries:
        dotproduct_vector = torch.Tensor([])
        for j in keys:
            dotproduct_vector = torch.cat([dotproduct_vector, torch.dot(i,j).reshape(-1)])
        dotproduct_matrix = torch.cat([dotproduct_matrix, dotproduct_vector.reshape(1,-1)])
     
    return dotproduct_matrix

In [44]:
def getSoftmaxed_qkdp(qk_dotproductmatrix):
    
    sm = nn.Softmax(dim = 0)
    sm_matrix = torch.tensor([])
    for i in qk_dotproductmatrix:
        sm_matrix = torch.cat([sm_matrix, sm(i).reshape(1,-1)])
        
    return sm_matrix
    

In [45]:
def getSoftmaxWeightedValues(softmaxed_qkdp, values):
    
    dim2_mat = torch.tensor([])
    dim3_mat = torch.tensor([])
    
    outer_loop_range = softmaxed_qkdp.shape[0]
    inner_loop_range = values.shape[0]
    
    
    for i in range(outer_loop_range):
        for j in range(inner_loop_range):
            dim2_mat = torch.cat([dim2_mat, (softmaxed_qkdp[i][j]*values[j]).reshape(-1)])
        dim3_mat = torch.cat([dim3_mat, dim2_mat.reshape(1,values.shape[0],values.shape[1])])
        dim2_mat = torch.tensor([])

        
    return dim3_mat

In [ ]:
'''
wordVecs = getWordVectors('Hi there this is nuts')
pos_encoded = PositionalEncoding(wordVecs)

new_dim = 3
queries, keys, values = qkvs(pos_encoded, new_dim)
qk_dotproductmatrix = qk_dotproducts(queries, keys)

d_k = keys.shape[1] # to be changed later to square root of 'key' vector dimension
qk_dotproductmatrix/=d_k

softmaxed_qkdp = getSoftmaxed_qkdp(qk_dotproductmatrix)
softmax_weighted_values = getSoftmaxWeightedValues(softmaxed_qkdp, values)

'''

In [55]:
wordVecs = getWordVectors('Hi there this is nuts')
# wordVecs

In [47]:
pos_encoded = PositionalEncoding(wordVecs)
# pos_encoded

In [48]:
queries, keys, values = qkvs(pos_encoded, new_dim)

In [49]:
qk_dotproductmatrix = qk_dotproducts(queries, keys)

In [50]:
d_k = keys.shape[1] # to be changed later to square root of 'key' vector dimension
qk_dotproductmatrix/=d_k

In [51]:
softmaxed_qkdp = getSoftmaxed_qkdp(qk_dotproductmatrix)

In [52]:
softmax_weighted_values = getSoftmaxWeightedValues(softmaxed_qkdp, values)

In [ ]:
softmax_weighted_values